<a href="https://colab.research.google.com/github/Junaidirah/predictive_pm25/blob/master/notebook/eda_gku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

In [9]:
# path = "D:\development\predictive_pm25\data\data_training\TUTL_RAW_2023-2025.csv"
path = "/content/GKU_RAW_2023-2025.csv"

In [10]:
df = pd.read_csv(path)

## EXPLORE DATA BEFORE NORMALIZE AND VALIDATION

In [7]:
df.info()

NameError: name 'df' is not defined

In [ ]:
duplicated_rows = df[df.duplicated()]
print(f"Number of duplicated rows: {len(duplicated_rows)}")

In [ ]:
df.describe()

In [ ]:
import calendar
df['created_at'] = pd.to_datetime(df['created_at'])
df_2023 = df[df['created_at'].dt.year == 2023].copy()
df_2023 = df_2023.set_index('created_at')

# Fix: Added numeric_only=True to skip non-numeric columns like 'date'
daily_avg = df_2023.resample('D').mean(numeric_only=True)
daily_avg = daily_avg.round(1)
kolom_target = 'pm25'

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 15), sharey=True)
fig.suptitle(f'Rata-rata Harian {kolom_target} Tahun 2023', fontsize=20)

axes = axes.flatten()

for i in range(1, 13):
    data_bulan = daily_avg[daily_avg.index.month == i]
    ax = axes[i-1]

    if not data_bulan.empty:
        ax.bar(data_bulan.index.day, data_bulan[kolom_target],
               color='skyblue', edgecolor='navy', label='Data Harian')

        rata_rata_bulan = data_bulan[kolom_target].mean()
        ax.axhline(y=rata_rata_bulan, color='red', linestyle='--', alpha=0.7, label='Rata-rata Sebulan')

    nama_bulan = calendar.month_name[i]
    ax.set_title(nama_bulan, fontsize=12, fontweight='bold')
    ax.set_xticks(range(1, 32, 5))
    ax.grid(axis='y', linestyle='--', alpha=0.5)

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=2, bbox_to_anchor=(0.5, 0.94), fontsize=14)
plt.tight_layout(rect=[0, 0.03, 1, 0.92])
plt.show()

### Data Cleaning


In [ ]:
total_rows = len(df)
missing = df.isnull().sum()
present = df.notnull().sum()
pct_missing = (missing / total_rows) * 100
quality_report = pd.DataFrame({
    'Data Hilang': missing,
    'Data Tersedia': present,
    'Total Seharusnya': total_rows,
    '% Hilang': pct_missing
})
print(quality_report.sort_values(by='% Hilang', ascending=False))

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['year'] = df['created_at'].dt.year
target_cols = ['pm25', 'temperature', 'humidity', 'ws', 'wd','sht31_temp','sht31_hum']

percentage_missing = df.groupby('year')[target_cols].apply(
    lambda x: x.isnull().mean() * 100
)

percentage_missing = percentage_missing.round(2)

percentage_missing = percentage_missing.reset_index()

print("Data Awal (Cuplikan):")
df.head()
print("\n--- Hasil Persentase Data Hilang per Tahun per Kolom ---")
print(percentage_missing)

In [ ]:
def get_days_in_year(year):
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        return 366
    return 365

# Menghitung data aktual per tahun
yearly_summary = df.groupby('year').size().reset_index(name='data_aktual')

# Menghitung data ekspektasi (720 data/hari)
yearly_summary['data_seharusnya'] = yearly_summary['year'].apply(lambda y: get_days_in_year(y) * 720)

# Menghitung selisih dan persentase kelengkapan
yearly_summary['selisih'] = yearly_summary['data_seharusnya'] - yearly_summary['data_aktual']
yearly_summary['persentase_kelengkapan'] = (yearly_summary['data_aktual'] / yearly_summary['data_seharusnya'] * 100).round(2)

print("--- Perbandingan Data Aktual vs Ekspektasi Per Tahun ---")
display(yearly_summary)

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['date'] = df['created_at'].dt.date
df['year'] = df['created_at'].dt.year
daily_counts = df.groupby('date').size().reset_index(name='actual_count')
EXPECTED_DAILY = 720
daily_counts['expected_count'] = EXPECTED_DAILY
daily_counts['missing_count'] = daily_counts['expected_count'] - daily_counts['actual_count']
daily_counts['completeness_pct'] = (daily_counts['actual_count'] / daily_counts['expected_count']) * 100
daily_counts['completeness_pct'] = daily_counts['completeness_pct'].round(2)

def categorize_status(pct):
    if pct >= 99: return 'Sangat Baik'
    elif pct >= 90: return 'Baik'
    elif pct >= 50: return 'Kurang'
    else: return 'Kritis'

daily_counts['status'] = daily_counts['completeness_pct'].apply(categorize_status)

print("--- Contoh Hasil Pengecekan Harian ---")
print(daily_counts.head())

def get_yearly_expected(year):
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        days = 366
    else:
        days = 365
    return days * 720
# Group by tahun
yearly_stats = df.groupby('year').size().reset_index(name='actual_total')
yearly_stats['expected_total'] = yearly_stats['year'].apply(get_yearly_expected)

# Hitung gap tahunan
yearly_stats['gap_total'] = yearly_stats['expected_total'] - yearly_stats['actual_total']
yearly_stats['completeness_pct'] = (yearly_stats['actual_total'] / yearly_stats['expected_total']) * 100
yearly_stats['completeness_pct'] = yearly_stats['completeness_pct'].round(2)

print("\n--- Rekap Kelengkapan Data Per Tahun ---")
print(yearly_stats)

# --- OPSIONAL: MELIHAT HARI DENGAN DATA PALING SEDIKIT ---
print("\n--- 5 Hari dengan Data Paling Sedikit (Terburuk) ---")
print(daily_counts.sort_values('actual_count').head(5))


In [ ]:
overload_days = daily_counts[daily_counts['actual_count'] > 720].sort_values('actual_count', ascending=False)

print(f"Ditemukan {len(overload_days)} hari dengan data berlebih:")

if not overload_days.empty:
    print(overload_days[['date', 'actual_count', 'missing_count', 'status']])
else:
    print("Sempurna. Tidak ada hari dengan data berlebih (double/spam).")

overload_days.head()

In [ ]:
# 1. Pastikan kolom tahun dan missing_count tersedia
daily_counts['year'] = pd.to_datetime(daily_counts['date']).dt.year
daily_counts['missing_count'] = 720 - daily_counts['actual_count']

# 2. Filter hanya hari-hari yang overload (> 720)
problematic_days = daily_counts[daily_counts['actual_count'] > 720].copy()

# 3. Hitung berapa banyak hari seperti itu di setiap tahun
summary_overload = problematic_days.groupby('year').agg(
    jumlah_hari_overload=('date', 'count'),
    total_data_berlebih=('missing_count', lambda x: abs(x).sum())
).reset_index()

# 4. Tambahkan perbandingan dengan total hari yang ada di data
total_days_per_year = daily_counts.groupby('year').size().reset_index(name='total_hari_terekam')
summary_overload = summary_overload.merge(total_days_per_year, on='year')

# 5. Hitung persentase kerusakan
summary_overload['persentase_hari_rusak'] = (summary_overload['jumlah_hari_overload'] / summary_overload['total_hari_terekam'] * 100).round(2)

print("--- Laporan Kerusakan Data Tahunan ---")
display(summary_overload)

In [ ]:
# 1. Persiapkan data harian dan kolom tahun
daily_pm25 = df.groupby('date')['pm25'].mean().reset_index()
daily_pm25['date'] = pd.to_datetime(daily_pm25['date'])
daily_pm25['year'] = daily_pm25['date'].dt.year
daily_pm25['month_name'] = daily_pm25['date'].dt.month_name()

# Urutan bulan agar rapi
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# 2. Visualisasi Box Plot per Tahun (2023, 2024, 2025)
years_to_plot = [2023, 2024, 2025]
fig, axes = plt.subplots(3, 1, figsize=(15, 18), sharex=True)

for i, year in enumerate(years_to_plot):
    data_year = daily_pm25[daily_pm25['year'] == year]
    if not data_year.empty:
        sns.boxplot(ax=axes[i], x='month_name', y='pm25', data=data_year,
                    order=month_order, hue='month_name', palette='viridis', legend=False)
        axes[i].set_title(f'Distribusi Rata-rata PM2.5 Harian - Tahun {year}', fontsize=14, fontweight='bold')
        axes[i].set_ylabel('PM2.5 (µg/m³)', fontsize=11)
        axes[i].grid(axis='y', linestyle='--', alpha=0.6)
    else:
        axes[i].set_title(f'Data Tahun {year} Tidak Tersedia', fontsize=14)

plt.xlabel('Bulan', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Menampilkan cuplikan dataframe utama (df) yang paling update
print(f"Ukuran dataframe saat ini: {df.shape}")
display(df.head())

In [ ]:
df['created_at']= pd.to_datetime(df['created_at'])
df_drop_2023 = df[df['created_at'].dt.year != 2023]
index_to_drop = df[df['created_at'].dt.year == 2023].index
df.drop(index_to_drop, inplace=True)
print("Data setelah menghapus tahun 2023:")
df.head()

## Normalize, cleaning, and validation data

In [ ]:
df.head()

In [ ]:
columns_to_drop = ['co2', 'pressure', 'solar', 'ws', 'wd']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
print(f"Columns dropped (or already missing): {columns_to_drop}")
df.head(10)

#### Validation Data

In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# SETUP & CONVERSION
# ==========================================

if 'created_at' in df.columns:
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.sort_values('created_at')
else:
    raise ValueError("Kolom 'created_at' tidak ditemukan!")

# Column mapping - UPDATED dengan sht31_temp dan sht31_hum
column_mapping = {
    'pm25': 'pm2_5',
    'temperature': 'temp',
    'humidity': 'hum',
    'sht31_temp': 'sht31_temp',      # BARU: SHT31 Temperature
    'sht31_hum': 'sht31_hum',        # BARU: SHT31 Humidity
}

# Batas Fisik (Hard Limit) - UPDATED
# sht31_temp dan sht31_hum memiliki range yang sama dengan temp dan hum
RANGE_LIMITS = {
    'pm2_5': (0, 250),           # PM2.5: 0-250 µg/m³
    'temp': (0, 80),             # Temperature: 0-80°C
    'hum': (0, 100),             # Humidity: 0-100%
    'sht31_temp': (0, 80),       # SHT31 Temp: 0-80°C (sama dengan temp)
    'sht31_hum': (0, 100),       # SHT31 Hum: 0-100% (sama dengan hum)
}

print("\n" + "=" * 70)
print("📊 PARAMETER KONFIGURASI")
print("=" * 70)
print("\n📋 Kolom yang akan divalidasi:")
for col, mapped_name in column_mapping.items():
    if col in df.columns:
        limit = RANGE_LIMITS.get(mapped_name, "N/A")
        print(f"   {col:15} → {mapped_name:12} | Range: {limit}")
    else:
        print(f"   {col:15} → {mapped_name:12} | ⚠️ TIDAK ADA DI DATA")

# Convert to numeric
for col_name in column_mapping.keys():
    if col_name in df.columns:
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        print(f"   ✓ {col_name} converted to numeric")

# ==========================================
# 2. KONFIGURASI
# ==========================================
EXPECTED_DATA_PER_HOUR = 30   # Asumsi data per 2 menit
THRESHOLD_PERCENTAGE = 0.75   # 75%
MIN_DATA_COUNT = int(EXPECTED_DATA_PER_HOUR * THRESHOLD_PERCENTAGE) # 22 Data

WINDOW_SIZE = 10
K_FACTOR = 1.5

print("\n" + "=" * 70)
print("⚙️ KONFIGURASI VALIDASI")
print("=" * 70)
print(f"Expected data per hour: {EXPECTED_DATA_PER_HOUR}")
print(f"Threshold percentage: {THRESHOLD_PERCENTAGE * 100}%")
print(f"Minimum data count: {MIN_DATA_COUNT} data/batch")
print(f"Sliding window size: {WINDOW_SIZE}")
print(f"K-factor (IQR multiplier): {K_FACTOR}")

# ==========================================
# 3. FUNGSI DETEKSI OUTLIER (Hard Limit + Statistik)
# ==========================================
def identify_outliers_sliding_window(data_series, window_size, k, limits=None):
    """
    Deteksi outlier dengan 2 tahap:
    1. Hard Limit: Cek range fisik
    2. Statistik: IQR sliding window
    """
    is_outlier = pd.Series(False, index=data_series.index)
    min_limit, max_limit = None, None

    # Unpack limit jika ada
    if limits:
        min_limit, max_limit = limits

    for i in range(len(data_series)):
        current_val = data_series.iloc[i]

        # Skip jika data kosong (NaN)
        if pd.isna(current_val):
            continue

        # --- TAHAP 1: Hard Limit (Validasi Range Fisik) ---
        if limits:
            if current_val < min_limit or current_val > max_limit:
                is_outlier.iloc[i] = True
                continue

        # --- TAHAP 2: Statistik Sliding Window ---
        window_start = max(0, i - window_size + 1)
        window_end = i + 1
        window_data = data_series.iloc[window_start:window_end].dropna()

        # Minimal 5 data valid untuk hitung statistik
        if len(window_data) > 5:
            Q1 = window_data.quantile(0.25)
            Q3 = window_data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - k * IQR
            upper_bound = Q3 + k * IQR

            if current_val < lower_bound or current_val > upper_bound:
                is_outlier.iloc[i] = True

    return is_outlier

# ==========================================
# 4. EKSEKUSI VALIDASI (Dynamic Time Window)
# ==========================================

print(f"\nTotal data awal: {len(df)}")
print(f"Syarat Kelengkapan: Minimal {MIN_DATA_COUNT} data per batch (75%).")
print("-" * 70)

valid_batches = []
incomplete_batches = []  # Untuk tracking batch yang tidak lengkap

# Grouper dengan origin='start'
grouper = pd.Grouper(key='created_at', freq='1h', origin='start')

for time_range, group_data in df.groupby(grouper):

    jumlah_data_aktual = len(group_data)

    if jumlah_data_aktual == 0:
        continue

    # Hitung waktu selesai batch
    end_time = time_range + pd.Timedelta(hours=1)

    # --- CEK A: KELENGKAPAN DATA (75%) ---
    if jumlah_data_aktual < MIN_DATA_COUNT:
        print(f"⚠️ [INCOMPLETE] Batch {time_range.time()} - {end_time.time()}: "
              f"{jumlah_data_aktual} data (Kurang dari 75%). "
              f"Data akan diubah menjadi NaN untuk imputation.")

        # Ubah SEMUA nilai dalam batch ini menjadi NaN
        group_data_nan = group_data.copy()

        # Ganti nilai dari kolom measured (bukan timestamp)
        # UPDATED: termasuk sht31_temp dan sht31_hum
        for col in ['pm25', 'temperature', 'humidity', 'sht31_temp', 'sht31_hum']:
            if col in group_data_nan.columns:
                group_data_nan[col] = np.nan

        incomplete_batches.append(group_data_nan)
        print(f"   └─ Seluruh nilai dalam batch diubah ke NaN")

        continue

    else:
        # --- CEK B: VALIDITAS NILAI (Range & Outlier) ---
        print(f"✅ [PROCESS] Batch {time_range.time()} - {end_time.time()}: "
              f"{jumlah_data_aktual} data. Checking value validity...")

        group_clean = group_data.copy()
        rows_to_drop_mask = pd.Series(False, index=group_clean.index)

        # UPDATED: loop melalui semua kolom termasuk sht31
        for col_df, key_limit in column_mapping.items():
            if col_df in group_clean.columns:
                if group_clean[col_df].isna().all():
                    continue

                limits = RANGE_LIMITS[key_limit]

                col_outliers = identify_outliers_sliding_window(
                    group_clean[col_df],
                    WINDOW_SIZE,
                    K_FACTOR,
                    limits=limits
                )

                rows_to_drop_mask = rows_to_drop_mask | col_outliers

        # Ubah baris yang invalid menjadi NaN (bukan dihapus)
        group_final = group_data.copy()

        # UPDATED: termasuk sht31_temp dan sht31_hum
        for col in ['pm25', 'temperature', 'humidity', 'sht31_temp', 'sht31_hum']:
            if col in group_final.columns:
                group_final.loc[rows_to_drop_mask, col] = np.nan

        valid_batches.append(group_final)

        deleted = rows_to_drop_mask.sum()
        print(f"   └─ {deleted} nilai (Out of range / Outlier) diubah ke NaN. "
              f"Sisa {(~rows_to_drop_mask).sum()} data valid.")

print("-" * 70)

# ==========================================
# 5. GABUNG DATA (Complete + Incomplete)
# ==========================================

all_batches = valid_batches + incomplete_batches

if all_batches:
    df_final_clean = pd.concat(all_batches, sort=False)
    df_final_clean = df_final_clean.sort_values('created_at')

    print("\n" + "=" * 70)
    print("📊 STATISTIK SETELAH VALIDASI (Sebelum Imputation)")
    print("=" * 70)

    print(f"\n✅ Data Lengkap (≥75%):  {len(valid_batches)} batch")
    print(f"⚠️  Data Tidak Lengkap (<75%): {len(incomplete_batches)} batch")

    total_rows = len(df_final_clean)
    # UPDATED: termasuk sht31_temp dan sht31_hum
    nan_count = df_final_clean[['pm25', 'temperature', 'humidity', 'sht31_temp', 'sht31_hum']].isna().sum()

    print(f"\n📋 Total Baris: {total_rows}")
    print(f"📊 NaN Count per kolom:")
    print(f"   PM2.5:       {nan_count.get('pm25', 0):3} NaN ({(nan_count.get('pm25', 0)/total_rows*100):5.2f}%)")
    print(f"   Temperature: {nan_count.get('temperature', 0):3} NaN ({(nan_count.get('temperature', 0)/total_rows*100):5.2f}%)")
    print(f"   Humidity:    {nan_count.get('humidity', 0):3} NaN ({(nan_count.get('humidity', 0)/total_rows*100):5.2f}%)")
    print(f"   SHT31 Temp:  {nan_count.get('sht31_temp', 0):3} NaN ({(nan_count.get('sht31_temp', 0)/total_rows*100):5.2f}%)")
    print(f"   SHT31 Hum:   {nan_count.get('sht31_hum', 0):3} NaN ({(nan_count.get('sht31_hum', 0)/total_rows*100):5.2f}%)")

    # ==========================================
    # 6. KOREKSI PM2.5 BERDASARKAN KELEMBABAN
    # ==========================================
    print("\n" + "=" * 70)
    print("🔧 TAHAP 6: KOREKSI PM2.5 BERDASARKAN KELEMBABAN")
    print("=" * 70)

    print("\n📌 Menggunakan humidity dari kolom 'humidity' untuk koreksi")
    print("   (Alternatif: dapat juga menggunakan sht31_hum jika humidity kosong)")

    # Hitung data sebelum koreksi (yang bukan NaN)
    valid_humidity_mask = df_final_clean['humidity'].notna()
    high_humidity_count = ((df_final_clean['humidity'] > 60) & valid_humidity_mask).sum()

    # Koreksi PM2.5 - HANYA untuk yang bukan NaN dan humidity > 60%
    correction_mask = (df_final_clean['humidity'] > 60) & (df_final_clean['pm25'].notna())
    df_final_clean.loc[correction_mask, 'pm25'] = \
        (df_final_clean.loc[correction_mask, 'pm25'] * 0.67).round(2)

    print(f"\n✅ Data dengan humidity > 60% yang dikoreksi: {correction_mask.sum()} baris")
    print(f"✅ Faktor koreksi: × 0.67")
    print(f"✅ Metode: Hanya koreksi nilai yang valid (bukan NaN)")

    # ==========================================
    # 7. REKAPITULASI AKHIR
    # ==========================================
    print("\n" + "=" * 70)
    print("📋 REKAPITULASI AKHIR")
    print("=" * 70)

    print(f"\nData Awal:              {len(df)} baris")
    print(f"Data Akhir:             {len(df_final_clean)} baris")
    print(f"Data Tertahan:          {len(df_final_clean)} baris (100%)")
    print(f"Data Hilang:            0 baris (0%)")
    print(f"Pendekatan:             NaN untuk missing/invalid values")

    print(f"\n📊 Persentase NaN (Missing Values):")
    total_nan = df_final_clean[['pm25', 'temperature', 'humidity', 'sht31_temp', 'sht31_hum']].isna().sum().sum()
    total_values = total_rows * 5
    print(f"   Total NaN: {total_nan} dari {total_values} values ({(total_nan/total_values*100):.2f}%)")
    print(f"   Siap untuk imputation dengan berbagai metode")

    print("\n🎯 OPSI IMPUTATION (Pilih salah satu):")
    print(f"   1. Forward Fill:     df.fillna(method='ffill')")
    print(f"   2. Backward Fill:    df.fillna(method='bfill')")
    print(f"   3. Interpolation:    df.interpolate(method='linear')")
    print(f"   4. Mean per Jam:     Isi dengan rata-rata jam tersebut")
    print(f"   5. Custom Method:    Kombinasi dari method di atas")

    print("\n" + "=" * 70)
    print("📋 SAMPLE OUTPUT (10 baris pertama)")
    print("=" * 70)
    display(df_final_clean.head(10))

    print("\n" + "=" * 70)
    print("📋 SAMPLE OUTPUT (Baris dengan NaN untuk imputation)")
    print("=" * 70)
    nan_sample = df_final_clean[df_final_clean['pm25'].isna()].head(10)
    if len(nan_sample) > 0:
        display(nan_sample)
    else:
        print("Tidak ada NaN di kolom PM2.5")

    # ==========================================
    # 8. DOKUMENTASI OPSI IMPUTATION
    # ==========================================
    print("\n" + "=" * 70)
    print("📚 OPSI IMPUTATION UNTUK LANGKAH SELANJUTNYA")
    print("=" * 70)

    print("""
REKOMENDASI: Gunakan Kombinasi Method untuk hasil optimal

# Method A: Simple (Linear Interpolation)
df_final_clean = df_final_clean.interpolate(method='linear', limit_direction='both')

# Method B: Advanced (Kombinasi - Recommended)
# Step 1: Linear interpolation untuk gap kecil
df_final_clean = df_final_clean.interpolate(method='linear', limit=10)

# Step 2: Mean per jam untuk gap menengah
for col in ['pm25', 'temperature', 'humidity', 'sht31_temp', 'sht31_hum']:
    df_final_clean[col] = df_final_clean.groupby(
        df_final_clean['created_at'].dt.hour
    )[col].transform(lambda x: x.fillna(x.mean()))

# Step 3: Forward fill untuk sisa
df_final_clean = df_final_clean.fillna(method='ffill')

# Step 4: Backward fill untuk awal dataset
df_final_clean = df_final_clean.fillna(method='bfill')

# Verify
print(f"Remaining NaN: {df_final_clean.isna().sum().sum()}")

# Save
df_final_clean.to_csv('data_clean_validated_sht31.csv', index=False)
    """)

    print("\n" + "=" * 70)
    print("✨ SELESAI! Data siap untuk imputation.")
    print("=" * 70)

else:
    print("❌ HASIL KOSONG.")
    print("Tidak ada batch data yang sesuai kriteria.")

print("\n" + "=" * 70)
print("📌 CATATAN PENTING")
print("=" * 70)
print("""
✅ Kolom yang divalidasi:
   - pm25 (PM2.5): Range 0-250 µg/m³
   - temperature: Range 0-80°C
   - humidity: Range 0-100%
   - sht31_temp: Range 0-80°C (sensor SHT31)
   - sht31_hum: Range 0-100% (sensor SHT31)

✅ Pendekatan NaN Handling:
   - Batch < 75% → Semua nilai jadi NaN
   - Outlier/Invalid → Nilai jadi NaN
   - Bukan dihapus → Data preserved

✅ Next Step:
   - Pilih method imputation dari opsi di atas
   - Apply imputation ke df_final_clean
   - Verify hasil reasonable
   - Save untuk analisis lanjutan

✅ Data Quality:
   - Tidak ada data yang dihapus (100% preserved)
   - Clear dokumentasi invalid values (NaN)
   - Siap untuk ML/Analysis
""")

### Valid Data

In [ ]:
df_final_clean.tail(10)

In [ ]:
total_rows = len(df_final_clean)
missing = df_final_clean.isnull().sum()
present = df_final_clean.notnull().sum()
pct_missing = (missing / total_rows) * 100
quality_report = pd.DataFrame({
    'Data Hilang': missing,
    'Data Tersedia': present,
    'Total Seharusnya': total_rows,
    '% Hilang': pct_missing
})
print(quality_report.sort_values(by='% Hilang', ascending=False))

In [ ]:
total_rows = len(df_final_clean)
rows_with_nan = df_final_clean.isna().any(axis=1).sum()
rows_without_nan = total_rows - rows_with_nan

print(f"Total baris: {total_rows}")
print(f"Baris yang mengandung setidaknya satu NaN: {rows_with_nan} ({(rows_with_nan/total_rows*100):.2f}%)")
print(f"Baris yang bersih (tanpa NaN): {rows_without_nan} ({(rows_without_nan/total_rows*100):.2f}%)")

In [ ]:
df_hourly = df_final_clean.set_index('created_at').resample('h').mean(numeric_only=True)
df_hourly = df_hourly.round(2)

print(f"Ukuran data setelah resample per jam: {df_hourly.shape}")
df_hourly.head(1000)

In [ ]:
import pandas as pd

# Memastikan df_hourly tersedia sebelum menghitung statistik
try:
    print("--- Statistik Nilai Kosong (NaN) per Kolom (df_hourly) ---")
    nan_summary = df_hourly.isna().sum()
    pct_nan = (df_hourly.isna().mean() * 100).round(2)

    summary_df = pd.DataFrame({
        'Jumlah NaN': nan_summary,
        'Persentase (%)': pct_nan
    })

    print(summary_df)
    print(f"\nTotal Baris: {len(df_hourly)}")
except NameError:
    print("Error: Variabel 'df_hourly' tidak ditemukan. Mohon jalankan kembali cell sebelumnya (mulai dari g7UuckRtS-d1).")

#### PM25 cleaning

In [ ]:
# Filter data for 2024 and create a copy
df_2024_plot = df_hourly[df_hourly.index.year == 2024].copy()

# Extract month names for the x-axis
df_2024_plot['month_name'] = df_2024_plot.index.month_name()

# Define standard month order
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Create the visualization
plt.figure(figsize=(15, 7))
sns.boxplot(x='month_name', y='pm25', data=df_2024_plot,
            order=month_order, hue='month_name', palette='viridis', legend=False)

plt.title('Distribusi Rata-rata PM2.5 Per Jam per Bulan - Tahun 2024', fontsize=14, fontweight='bold')
plt.xlabel('Bulan', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Menangani outlier menggunakan metode Monthly IQR Clipping (Per Year & Month)
# Kita akan membatasi nilai agar tepat berada di batas whisker masing-masing kelompok tahun-bulan

def cap_outliers_specific(group):
    if group['pm25'].dropna().empty:
        return group

    Q1 = group['pm25'].quantile(0.25)
    Q3 = group['pm25'].quantile(0.75)
    IQR = Q3 - Q1
    upper_whisker = Q3 + 1.5 * IQR
    lower_whisker = Q1 - 1.5 * IQR

    # Lakukan capping ke batas whisker spesifik kelompok ini
    group['pm25'] = group['pm25'].clip(lower=lower_whisker, upper=upper_whisker)
    return group

# Tambahkan kolom pembantu untuk grouping yang lebih detail (Tahun dan Bulan)
df_hourly['temp_year'] = df_hourly.index.year
df_hourly['temp_month'] = df_hourly.index.month

# Terapkan fungsi capping per (Tahun, Bulan)
df_hourly = df_hourly.groupby(['temp_year', 'temp_month'], group_keys=False).apply(cap_outliers_specific)

# Hapus kolom pembantu
df_hourly.drop(columns=['temp_year', 'temp_month'], inplace=True)

# --- Visualisasi ulang Tahun 2024 ---
df_2024_plot = df_hourly[df_hourly.index.year == 2024].copy()
df_2024_plot['month_name'] = df_2024_plot.index.month_name()

plt.figure(figsize=(15, 7))
sns.boxplot(x='month_name', y='pm25', data=df_2024_plot,
            order=month_order, hue='month_name', palette='viridis', legend=False)

plt.title('Distribusi PM2.5 Per Jam (Cleaned - No Outliers) - Tahun 2024', fontsize=14, fontweight='bold')
plt.ylabel('PM2.5 (µg/m³)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Filter data for 2025 and create a copy
df_2025_plot = df_hourly[df_hourly.index.year == 2025].copy()

# Extract month names for the x-axis
df_2025_plot['month_name'] = df_2025_plot.index.month_name()

# Define standard month order
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Create the visualization
plt.figure(figsize=(15, 7))
sns.boxplot(x='month_name', y='pm25', data=df_2025_plot,
            order=month_order, hue='month_name', palette='viridis', legend=False)

plt.title('Distribusi Rata-rata PM2.5 Per Jam per Bulan - Tahun 2025', fontsize=14, fontweight='bold')
plt.xlabel('Bulan', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Menangani outlier menggunakan metode Monthly IQR Clipping (Per Year & Month)
# Kita akan membatasi nilai agar tepat berada di batas whisker masing-masing kelompok tahun-bulan

def cap_outliers_specific(group):
    if group['pm25'].dropna().empty:
        return group

    Q1 = group['pm25'].quantile(0.25)
    Q3 = group['pm25'].quantile(0.75)
    IQR = Q3 - Q1
    upper_whisker = Q3 + 1.5 * IQR
    lower_whisker = Q1 - 1.5 * IQR

    # Lakukan capping ke batas whisker spesifik kelompok ini
    group['pm25'] = group['pm25'].clip(lower=lower_whisker, upper=upper_whisker)
    return group

# Tambahkan kolom pembantu untuk grouping yang lebih detail (Tahun dan Bulan)
df_hourly['temp_year'] = df_hourly.index.year
df_hourly['temp_month'] = df_hourly.index.month

# Terapkan fungsi capping per (Tahun, Bulan) -> Ini otomatis mencakup 2024 dan 2025
df_hourly = df_hourly.groupby(['temp_year', 'temp_month'], group_keys=False).apply(cap_outliers_specific)

# Hapus kolom pembantu
df_hourly.drop(columns=['temp_year', 'temp_month'], inplace=True)

# --- Visualisasi ulang Tahun 2024 dan 2025 ---
for year in [2024, 2025]:
    df_plot = df_hourly[df_hourly.index.year == year].copy()
    if not df_plot.empty:
        df_plot['month_name'] = df_plot.index.month_name()

        plt.figure(figsize=(15, 6))
        sns.boxplot(x='month_name', y='pm25', data=df_plot,
                    order=month_order, hue='month_name', palette='viridis', legend=False)

        plt.title(f'Distribusi PM2.5 Per Jam (Cleaned) - Tahun {year}', fontsize=14, fontweight='bold')
        plt.ylabel('PM2.5 (µg/m³)')
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

### TRH Cleaning

In [ ]:
# Visualisasi Distribusi Temperature dan Humidity per Jam (2024 & 2025)
for year in [2024, 2025]:
    df_plot = df_hourly[df_hourly.index.year == year].copy()
    if not df_plot.empty:
        df_plot['month_name'] = df_plot.index.month_name()

        # 1. Box Plot Temperature
        plt.figure(figsize=(15, 6))
        sns.boxplot(x='month_name', y='temperature', data=df_plot,
                    order=month_order, hue='month_name', palette='YlOrRd', legend=False)
        plt.title(f'Distribusi Temperature Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        plt.ylabel('Temperature (°C)')
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

        # 2. Box Plot Humidity
        plt.figure(figsize=(15, 6))
        sns.boxplot(x='month_name', y='humidity', data=df_plot,
                    order=month_order, hue='month_name', palette='GnBu', legend=False)
        plt.title(f'Distribusi Humidity Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        plt.ylabel('Humidity (%)')
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

In [ ]:
def cap_outliers_all_cols(group, columns):
    for col in columns:
        if group[col].dropna().empty:
            continue

        Q1 = group[col].quantile(0.25)
        Q3 = group[col].quantile(0.75)
        IQR = Q3 - Q1
        upper_whisker = Q3 + 1.5 * IQR
        lower_whisker = Q1 - 1.5 * IQR

        group[col] = group[col].clip(lower=lower_whisker, upper=upper_whisker)
    return group

# Tambahkan kolom pembantu
df_hourly['temp_year'] = df_hourly.index.year
df_hourly['temp_month'] = df_hourly.index.month

# Terapkan capping untuk temperature dan humidity
target_cols = ['temperature', 'humidity']
df_hourly = df_hourly.groupby(['temp_year', 'temp_month'], group_keys=False).apply(
    lambda x: cap_outliers_all_cols(x, target_cols)
)

# Hapus kolom pembantu
df_hourly.drop(columns=['temp_year', 'temp_month'], inplace=True)

# --- Visualisasi Hasil Pembersihan (2024 & 2025) ---
for year in [2024, 2025]:
    df_plot = df_hourly[df_hourly.index.year == year].copy()
    if not df_plot.empty:
        df_plot['month_name'] = df_plot.index.month_name()

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

        # 1. Box Plot Temperature
        sns.boxplot(ax=ax1, x='month_name', y='temperature', data=df_plot,
                    order=month_order, hue='month_name', palette='YlOrRd', legend=False)
        ax1.set_title(f'Cleaned Temperature Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Temperature (°C)')
        ax1.grid(axis='y', linestyle='--', alpha=0.7)

        # 2. Box Plot Humidity
        sns.boxplot(ax=ax2, x='month_name', y='humidity', data=df_plot,
                    order=month_order, hue='month_name', palette='GnBu', legend=False)
        ax2.set_title(f'Cleaned Humidity Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Humidity (%)')
        ax2.grid(axis='y', linestyle='--', alpha=0.7)

        plt.tight_layout()
        plt.show()

#### SHT31_temp SHT31_hum

In [ ]:
# Visualisasi Distribusi SHT31 Temp dan Hum per Jam (2024 & 2025)
for year in [2024, 2025]:
    df_plot = df_hourly[df_hourly.index.year == year].copy()
    if not df_plot.empty:
        df_plot['month_name'] = df_plot.index.month_name()

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

        # 1. Box Plot SHT31 Temperature
        sns.boxplot(ax=ax1, x='month_name', y='sht31_temp', data=df_plot,
                    order=month_order, hue='month_name', palette='Oranges', legend=False)
        ax1.set_title(f'Distribusi SHT31 Temperature Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        ax1.set_ylabel('SHT31 Temp (°C)')
        ax1.grid(axis='y', linestyle='--', alpha=0.7)

        # 2. Box Plot SHT31 Humidity
        sns.boxplot(ax=ax2, x='month_name', y='sht31_hum', data=df_plot,
                    order=month_order, hue='month_name', palette='Blues', legend=False)
        ax2.set_title(f'Distribusi SHT31 Humidity Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        ax2.set_ylabel('SHT31 Hum (%)')
        ax2.grid(axis='y', linestyle='--', alpha=0.7)

        plt.tight_layout()
        plt.show()

In [ ]:
def cap_outliers_all_cols(group, columns):
    for col in columns:
        if group[col].dropna().empty:
            continue

        Q1 = group[col].quantile(0.25)
        Q3 = group[col].quantile(0.75)
        IQR = Q3 - Q1
        upper_whisker = Q3 + 1.5 * IQR
        lower_whisker = Q1 - 1.5 * IQR

        group[col] = group[col].clip(lower=lower_whisker, upper=upper_whisker)
    return group

# Tambahkan kolom pembantu
df_hourly['temp_year'] = df_hourly.index.year
df_hourly['temp_month'] = df_hourly.index.month

# Terapkan capping untuk SHT31 temperature dan humidity
target_cols_sht31 = ['sht31_temp', 'sht31_hum']
df_hourly = df_hourly.groupby(['temp_year', 'temp_month'], group_keys=False).apply(
    lambda x: cap_outliers_all_cols(x, target_cols_sht31)
)

# Hapus kolom pembantu
df_hourly.drop(columns=['temp_year', 'temp_month'], inplace=True)

# --- Visualisasi Hasil Pembersihan SHT31 (2024 & 2025) ---
for year in [2024, 2025]:
    df_plot = df_hourly[df_hourly.index.year == year].copy()
    if not df_plot.empty:
        df_plot['month_name'] = df_plot.index.month_name()

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

        # 1. Box Plot SHT31 Temperature
        sns.boxplot(ax=ax1, x='month_name', y='sht31_temp', data=df_plot,
                    order=month_order, hue='month_name', palette='Oranges', legend=False)
        ax1.set_title(f'Cleaned SHT31 Temperature Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        ax1.set_ylabel('SHT31 Temp (°C)')
        ax1.grid(axis='y', linestyle='--', alpha=0.7)

        # 2. Box Plot SHT31 Humidity
        sns.boxplot(ax=ax2, x='month_name', y='sht31_hum', data=df_plot,
                    order=month_order, hue='month_name', palette='Blues', legend=False)
        ax2.set_title(f'Cleaned SHT31 Humidity Per Jam - Tahun {year}', fontsize=14, fontweight='bold')
        ax2.set_ylabel('SHT31 Hum (%)')
        ax2.grid(axis='y', linestyle='--', alpha=0.7)

        plt.tight_layout()
        plt.show()